In [1]:
import tensorflow as tf
from tensorflow import keras

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
print(f"Tf: {tf.__version__}\nKeras: {keras.__version__}")

Tf: 2.12.0
Keras: 2.12.0


In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/An toàn bảo mật hệ thống thông tin

/content/drive/MyDrive/An toàn bảo mật hệ thống thông tin


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

#thư viện cho phép replace giá trị trong numpy bằng giá trị ta muốn
from sklearn.impute import SimpleImputer

#thư viện cho phép transform dữ liệu category thành số
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

#thư viện cho pháp transofrm dữ liệu label yes/no thành số, thường dùng cho true/false
from sklearn.preprocessing import LabelEncoder

#thư viện cho phép tách dataset thành train test
from sklearn.model_selection import train_test_split

#thư viện cho phép feature scaling
from sklearn.preprocessing import StandardScaler

#thư viện cho phép làm viện trong đa luồng
from sklearn.pipeline import Pipeline

#thư viện cho phép đánh giá model
from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report, accuracy_score,f1_score

#thư viện cho phép cross validation
from sklearn.model_selection import cross_val_score

#thư viện cho phép đọc folder
import os

In [ ]:
# Set the path to the folder containing the CSV files
train_dataset_fp = './Dataset/phishing.csv'

In [ ]:
train_df = pd.read_csv(train_dataset_fp)

In [ ]:
train_df


,Index,UsingIP,LongURL,ShortURL,Symbol@,Redirecting//,PrefixSuffix-,SubDomains,HTTPS,DomainRegLen,...,UsingPopupWindow,IframeRedirection,AgeofDomain,DNSRecording,WebsiteTraffic,PageRank,GoogleIndex,LinksPointingToPage,StatsReport,class
0,0,1,1,1,1,1,-1,0,1,-1,...,1,1,-1,-1,0,-1,1,1,1,-1
1,1,1,0,1,1,1,-1,-1,-1,-1,...,1,1,1,-1,1,-1,1,0,-1,-1
2,2,1,0,1,1,1,-1,-1,-1,1,...,1,1,-1,-1,1,-1,1,-1,1,-1
3,3,1,0,-1,1,1,-1,1,1,-1,...,-1,1,-1,-1,0,-1,1,1,1,1
4,4,-1,0,-1,1,-1,-1,1,1,-1,...,1,1,1,1,1,-1,1,-1,-1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11049,11049,1,-1,1,-1,1,1,1,1,-1,...,-1,-1,1,1,-1,-1,1,1,1,1
11050,11050,-1,1,1,-1,-1,-1,1,-1,-1,...,-1,1,1,1,1,1,1,-1,1,-1
11051,11051,1,-1,1,1,1,-1,1,-1,-1,...,1,1,1,1,1,-1,1,0,1,-1
11052,11052,-1,-1,1,1,1,-1,-1,-1,1,...,-1,1,1,1,1,-1,1,1,1,-1


In [ ]:
train_df.columns

Index(['Index', 'UsingIP', 'LongURL', 'ShortURL', 'Symbol@', 'Redirecting//',
       'PrefixSuffix-', 'SubDomains', 'HTTPS', 'DomainRegLen', 'Favicon',
       'NonStdPort', 'HTTPSDomainURL', 'RequestURL', 'AnchorURL',
       'LinksInScriptTags', 'ServerFormHandler', 'InfoEmail', 'AbnormalURL',
       'WebsiteForwarding', 'StatusBarCust', 'DisableRightClick',
       'UsingPopupWindow', 'IframeRedirection', 'AgeofDomain', 'DNSRecording',
       'WebsiteTraffic', 'PageRank', 'GoogleIndex', 'LinksPointingToPage',
       'StatsReport', 'class'],
      dtype='object')

In [ ]:
# column order in CSV file
column_names = ['Index', 'UsingIP', 'LongURL', 'ShortURL', 'Symbol@', 'Redirecting//',
       'PrefixSuffix-', 'SubDomains', 'HTTPS', 'DomainRegLen', 'Favicon',
       'NonStdPort', 'HTTPSDomainURL', 'RequestURL', 'AnchorURL',
       'LinksInScriptTags', 'ServerFormHandler', 'InfoEmail', 'AbnormalURL',
       'WebsiteForwarding', 'StatusBarCust', 'DisableRightClick',
       'UsingPopupWindow', 'IframeRedirection', 'AgeofDomain', 'DNSRecording',
       'WebsiteTraffic', 'PageRank', 'GoogleIndex', 'LinksPointingToPage',
       'StatsReport', 'class']

feature_names = column_names[1:-1]
label_name = column_names[-1]

print("Features: {}".format(feature_names))
print("Label: {}".format(label_name))

Features: ['UsingIP', 'LongURL', 'ShortURL', 'Symbol@', 'Redirecting//', 'PrefixSuffix-', 'SubDomains', 'HTTPS', 'DomainRegLen', 'Favicon', 'NonStdPort', 'HTTPSDomainURL', 'RequestURL', 'AnchorURL', 'LinksInScriptTags', 'ServerFormHandler', 'InfoEmail', 'AbnormalURL', 'WebsiteForwarding', 'StatusBarCust', 'DisableRightClick', 'UsingPopupWindow', 'IframeRedirection', 'AgeofDomain', 'DNSRecording', 'WebsiteTraffic', 'PageRank', 'GoogleIndex', 'LinksPointingToPage', 'StatsReport']
Label: class


In [ ]:
batch_size = 32
class_names = ['-1', '1']
train_dataset = tf.data.experimental.make_csv_dataset(
    train_dataset_fp,
    batch_size,
    column_names=column_names,
    label_name=label_name,
    num_epochs=1)

In [ ]:
def map_fn(features, label):
    modified_label = tf.where(tf.equal(label, -1), tf.zeros_like(label), label)
    return features, modified_label

train_dataset = train_dataset.map(map_fn)

In [ ]:
features, labels = next(iter(train_dataset))

print(features)

OrderedDict([('Index', <tf.Tensor: shape=(32,), dtype=int32, numpy=
array([7131, 6995, 8508, 9001,   82, 6715, 9386, 2723, 2499, 7544, 8866,
       3186, 9136, 1584, 6718, 9648, 5314, 7829, 9413, 7012, 1708, 9046,
       6908, 1159, 8081, 9490, 2216, 5392, 8178, 5113, 6222, 1930],
      dtype=int32)>), ('UsingIP', <tf.Tensor: shape=(32,), dtype=int32, numpy=
array([-1,  1, -1, -1,  1,  1, -1, -1,  1, -1, -1,  1, -1,  1,  1, -1, -1,
        1, -1, -1,  1,  1, -1,  1,  1, -1,  1, -1,  1,  1,  1,  1],
      dtype=int32)>), ('LongURL', <tf.Tensor: shape=(32,), dtype=int32, numpy=
array([-1, -1,  1,  1,  1, -1, -1,  1,  1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1,  1,  1, -1, -1, -1, -1, -1, -1, -1,  1,  1, -1, -1],
      dtype=int32)>), ('ShortURL', <tf.Tensor: shape=(32,), dtype=int32, numpy=
array([ 1,  1,  1,  1,  1,  1,  1, -1, -1,  1,  1,  1,  1,  1, -1,  1,  1,
        1, -1,  1,  1,  1,  1,  1,  1,  1,  1, -1, -1, -1,  1,  1],
      dtype=int32)>), ('Symbol@', <tf.Tensor: shape

In [ ]:
def pack_features_vector(features, labels):
  """Pack the features into a single array."""
  features = tf.stack(list(features.values()), axis=1)
  return features, labels

In [ ]:
train_dataset = train_dataset.map(pack_features_vector)


In [ ]:
features, labels = next(iter(train_dataset))

print(features)

In [ ]:
# Xây dựng mô hình bao gồm
# 2 tầng ẩn mỗi tầng có 10 nơ ron + activation là relu
# Lớp Softmax phân loại thành 3 nhãn

model = tf.keras.Sequential([
  tf.keras.layers.Dense(10, activation=tf.nn.relu, input_shape=(31,)),  # input shape required
  tf.keras.layers.Dense(10, activation=tf.nn.relu),
  tf.keras.layers.Dropout(0.2) ,
  tf.keras.layers.Dense(2)
])

In [ ]:
#Định nghĩa hàm mất mát
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)


In [ ]:
def loss(model, x, y, training):
  # training=training is needed only if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
  y_ = model(x, training=training)
  return loss_object(y_true=y, y_pred=y_)


l = loss(model, features, labels, training=False)
print("Loss test: {}".format(l))

Loss test: 870.1578369140625


In [ ]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)

In [ ]:
model.trainable_variables

In [ ]:
def grad(model, inputs, targets):
  with tf.GradientTape() as tape:
    loss_value = loss(model, inputs, targets, training=True)
  return loss_value, tape.gradient(loss_value, model.trainable_variables)

In [ ]:
#Thử nghiệm quá trình này

loss_value, grads = grad(model, features, labels)

print("Step: {}, Initial Loss: {}".format(optimizer.iterations.numpy(),
                                          loss_value.numpy()))

optimizer.apply_gradients(zip(grads, model.trainable_variables))

print("Step: {},         Loss: {}".format(optimizer.iterations.numpy(),
                                          loss(model, features, labels, training=True).numpy()))

Step: 0, Initial Loss: 903.301513671875


<tf.Variable 'UnreadVariable' shape=() dtype=int64, numpy=1>

Step: 1,         Loss: 12862.6923828125


In [ ]:
#Tiến hành training

train_loss_results = []
train_accuracy_results = []

num_epochs = 201

for epoch in range(num_epochs):
  epoch_loss_avg = tf.keras.metrics.Mean()
  epoch_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

  # lặp qua các batch và đổ vào mô hình
  for x, y in train_dataset:
    # Tính gradient
    loss_value, grads = grad(model, x, y)

    # Cập nhật tham số của mô hình thông qua Gradient
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    # Tính giá trị sai lệch
    epoch_loss_avg.update_state(loss_value)  # Add current batch loss

    # Tính độ chính xác
    epoch_accuracy.update_state(y, model(x, training=True))

  # End epoch
  train_loss_results.append(epoch_loss_avg.result())
  train_accuracy_results.append(epoch_accuracy.result())

  if epoch % 50 == 0:
    print("Epoch {:03d}: Loss: {:.3f}, Accuracy: {:.3%}".format(epoch,
                                                                epoch_loss_avg.result(),
                                                                epoch_accuracy.result()))

In [ ]:
fig, axes = plt.subplots(2, sharex=True, figsize=(12, 8))
fig.suptitle('Training Metrics')

axes[0].set_ylabel("Loss", fontsize=14)
axes[0].plot(train_loss_results)

axes[1].set_ylabel("Accuracy", fontsize=14)
axes[1].set_xlabel("Epoch", fontsize=14)
axes[1].plot(train_accuracy_results)
plt.show()

In [ ]:
#Hiển thị quá trình training
fig, axes = plt.subplots(2, sharex=True, figsize=(12, 8))
fig.suptitle('Training Metrics')

axes[0].set_ylabel("Loss", fontsize=14)
axes[0].plot(train_loss_results)

axes[1].set_ylabel("Accuracy", fontsize=14)
axes[1].set_xlabel("Epoch", fontsize=14)
axes[1].plot(train_accuracy_results)
plt.show()